# A simple One-Shot DMET code for 1D Hubbard-Like model

Chong Sun

We present a very simple DMET python code for Fermionic lattice models. 
The purpose of this notebook is to explain the algorithm. The idea of DMET algorithm is shown as follows

<img src="./images/DMET.png" alt="DMET Algorithm" width="700"/>

<!-- <img src="./images/DMET_flowchart.png" alt="DMET Algorithm flowchart" width="400"/> -->

In this one-shot DMET, we will not construct the DMET loop to optimize the correlation potential, instead, our goal is to **find the next correlation potetial**.

There are many different numerical choices when implementing DMET, and you can find them in this paper: [Wouters et al. JCTC, 12, 6, 2706-2719](https://pubs.acs.org/doi/10.1021/acs.jctc.6b00316). Here we stick to **one** implementation, so you don't get confused!


In [1]:
import numpy as np
from pyscf import fci 
from scipy.optimize import minimize # for chem potential minimization

## Setting up the system
We need the information for both (1) the whole lattice and (2) the impurity. 
Here, we only consider systems with **translational symmetry** and **periodic boundary conditions (PBC)**. 
We use the a Hubbard-like Hamiltonian as an example:
$$H = -t\sum_{i=1}^N (a^\dagger_i a_{i+1} + \text{h.c.}) + U\sum_{i=1}^N n_i n_{i+1}$$
where $N$ is the number of sites (or orbitals in the chem language), and $n_i = a^\dagger_i a_i$ is the number operator. We set $N+1 = 0$ to enforce PBC. The one-body term represents electron hopping between adjacent sites with amplitude $-t$, and the two-body term represents Columb repulsion between adjacent sites with strength $U$.

For simplicity, we will not consider systems with different spins. We assume there are only up spins in the lattice.
### Impurity (fragment) setup
We pick the first $N_\text{imp}$ sites as the impurity (fragment, cell), and the rest is the environment. 
Due to translational symmetry, the impurity shares the same property as the second, third ... $N_\text{imp}$ sites, which makes it easy to evaluate the properties of the whole system. E.g.,
$$E_\text{tot} = \frac{N}{N_\text{imp}} E_\text{imp}$$
Of course, $E_\text{tot}$ is only a good approximation, but not the exact total energy, because the embedding is not exact! 

In [2]:
# Setting up the system
norb = 6 # number of orbitals in the lattice
nimp = 2 # number of impurity orbitals (fragments)
ncell = norb // nimp # number of cells 
nemb = nimp*2 # embedding space is always twice of impurity space for ground state
nelec = norb//2 # number of total electrons
ne_emb = nimp # number of total electrons in the embedding space TODO, use the environmental RDM to generate 
ne_imp = nelec // ncell # due to translational invariance, the number of electrons is evenly distributed
U = 4 # setting the interacting strength

In [10]:
# Hamiltonian matrices
h1e = np.zeros((norb, norb)) 
h2e = np.zeros((norb,)*4) # order: H2 = h2e_{pq,rs} p^+ q r^+ s = (pq|rs) p^+ r^+ s q - (pq|rs) \delta_{qr} p^+ s
for i in range(norb):
    h1e[i, (i+1)%norb] = -1
    h1e[i, (i-1)%norb] = -1
    h2e[i,i,(i+1)%norb,(i+1)%norb] = U/2 
    h2e[(i+1)%norb,(i+1)%norb, i, i] = U/2 # restoring the symmetry of real-valued h2e: V_{ij, kl} = V_{kl, ij} = V_{ji, lk} = V_{lk, ji}

## Correlation potential
In order to mimic the two-body interactions from the environment, we add a correlation potential $u$ to the impurity. 

In [23]:
# Initialize correlation potential
n_param = nimp*(nimp+1)//2
u_params0 = np.zeros(n_param) # because the correlation potential is hermitian, we only need the upper triangle
# u_params0 = np.array([-0.38445013,  0.00963527, -0.00047456])
def params2ucorr(params): 
    '''Turn parameters into the correlation potential matrix'''
    u_corr = np.zeros((nimp, nimp), dtype=params.dtype)
    iu = np.triu_indices(nimp)
    u_corr[iu] = params
    u_corr += u_corr.T
    np.fill_diagonal(u_corr, np.diagonal(u_corr)/2)
    return u_corr
u_corr0 = params2ucorr(u_params0)

def imp2latt_1e(m_imp):
    '''Impurity to lattice '''
    return np.kron(np.eye(ncell), m_imp)


In [24]:
# add correlation potential to the 1-body Hamiltonian
def add_u_corr(h1e, u_corr, nimp):
    n_cell = norb // nimp 
    h1e_new = np.copy(h1e)
    for x in range(n_cell):
        h1e_new[x*nimp:(x+1)*nimp, x*nimp:(x+1)*nimp] += u_corr 
    return h1e_new 

h1e_new = add_u_corr(h1e, u_corr0, nimp)

In [25]:
# Solve low-level Hamiltonian and evaluate the rdm1
mo_e, mo_orbs = np.linalg.eigh(h1e_new)
orb_occ = mo_orbs[:, :nelec]
rdm1 = np.dot(orb_occ, orb_occ.T)

In [26]:
# construct bath from rdm1
rdm1_mix = rdm1[nimp:, :nimp]
bath, s, _ = np.linalg.svd(rdm1_mix, full_matrices=False) # s should show how correlated A and E are

In [27]:
# construct rotation (projection) matrix to the embedding space
rotmat = np.zeros((norb, nimp*2))
rotmat[:nimp, :nimp] = np.eye(nimp)
rotmat[nimp:, nimp:] = bath 

In [28]:
# construct embedding Hamiltonian with interacting bath formulation
h1e_emb = np.dot(np.dot(rotmat.T.conj(), h1e), rotmat) 
h2e_emb = np.einsum('pi, qj, pqrs, rk, sl ->ijkl', rotmat.conj(), rotmat.conj(), h2e, rotmat, rotmat) # using pyscf tradition c^+ c^+ c c


In [29]:
# fci solver for spinless case (only have alpha electrons)

def fci_spinless(h1e, eri, norb, nelec):
    nelec = (nelec, 0)
    cisolver = fci.direct_spin1
    e, c = cisolver.kernel(h1e, eri, norb, nelec)
    rdm1 = cisolver.make_rdm1(c, norb, nelec)
    return e, rdm1 

In [30]:
# add a global chemical potential to make sure the impurity electron  = ne_imp
# we need to optimize the chemical potential
mu_global0 = 1.0
def diff_ne_imp(mu):
    mu_glob_mat = np.zeros((nemb, nemb))
    mu_glob_mat[:nimp, :nimp] = np.eye(nimp)*mu
    h1e_emb_new = h1e_emb - mu_glob_mat
    _, rdm1_emb = fci_spinless(h1e_emb_new, h2e_emb, 2*nimp, ne_emb)
    ne_imp_n = np.sum(np.diag(rdm1_emb)[:nimp])
    ne_diff = abs(ne_imp_n - ne_imp)
    #print(f"Electron number diff: {ne_diff}")
    return ne_diff

# optimize mu_global to minimize diff_ne_imp
mu_global = minimize(diff_ne_imp, x0=mu_global0, method="Powell", tol=1e-3, options={'maxiter': 3}).x
print(f"Optimized global chemical potential (mu_global): {mu_global}")

Optimized global chemical potential (mu_global): [0.63999993]


In [31]:
#solve the embedding Hamiltonian
mu_glob_mat = np.zeros((nemb, nemb))
mu_glob_mat[:nimp, :nimp] = np.eye(nimp)*mu_global
h1e_emb_new = h1e_emb - mu_glob_mat
e, rdm1_emb = fci_spinless(h1e_emb_new, h2e_emb, 2*nimp, ne_emb)

In [32]:
# Next compare the difference between the two RDM1s
rdm1_latt_emb = np.dot(np.dot(rotmat.T, rdm1), rotmat)
# We only compare the impurity part of rdm1
rdm1_tol = 1e-5
rdm1_imp_diff = rdm1_emb[:nimp, :nimp] - rdm1_latt_emb[:nimp, :nimp] 
abs_rdm1_imp_diff = (rdm1_imp_diff**2).sum()
if abs_rdm1_imp_diff < rdm1_tol:
    print("RDM1 are the same, DMET converged")
else:
    print(f"RDM1 diff = {abs_rdm1_imp_diff}, we need to change the correlation potential!")

RDM1 diff = 0.0015246425788277142, we need to change the correlation potential!


## Deriving the gradient for optimizing the correlation potential
We want to minimize
$$f(u) = ||D^\text{emb}_{\text{FCI}} - D^\text{emb}_{\text{HF}}(u)||^2 = \sum_{ij}[D^\text{emb}_{\text{FCI}} - D^\text{emb}_{\text{HF}}(u)]_{ij}^2$$
with respect to $u$.

We derive the analytic gradient $\rm{d} f/\rm{d} u$. 
$$\frac{\rm{d} f}{\rm{d} u} = -2\sum_{ij}(D^\text{emb}_{\text{FCI}} - D^\text{emb}_{\text{HF}}(u))_{ij}\frac{\rm{d} D^\text{emb}_{\text{HF}}(u)}{\rm{d} u}_{ij} $$
where $D^\text{emb}_{\text{HF}}(u) = R^\dagger D_{\text{HF}}(u) R$ with $R$ being the projection operator to the embedding space. 

Next we evaluate $\frac{\rm{d} D_{\text{HF}}(u)}{\rm{d} u}$ using first-order perturbation theory. Let
$$H = H^0 + \delta H^1$$
where $H^0$ and $H^1$ are both 1-body Hamiltonians, and
$$H \begin{bmatrix}C_\text{occ} & C_\text{vir} \end{bmatrix} = \begin{bmatrix}C_\text{occ} & C_\text{vir} \end{bmatrix} \begin{bmatrix}E_\text{occ} & \\ & E_\text{vir}\end{bmatrix}$$
Therefore, $D_{\text{HF}}$ is evaluated as 
$$D_{\text{HF}} = ( C^0_\text{occ} + \delta C^1_\text{occ})( C^0_\text{occ} + \delta C^1_\text{occ})^\dagger$$
And
$$\left.\frac{\partial D_{\text{HF}}}{\partial \delta }\right\vert_{\delta=0} = C^0_\text{occ}C^{1,\dagger}_\text{occ} + C^1_\text{occ}C^{0,\dagger}_\text{occ} + O(\delta^2)$$

#### Evaluating $C^1_\text{occ}$

Substituting $H = H^0 + \delta H^1$, $C = C^0 + \delta C^1$ and $E = E^0 + \delta E^1$ into the Hamiltonian equation, and only take the first order of $\delta$, we get 
$$H^0 C^1_\text{occ} + H^1 C^0_\text{occ} = C^1_\text{occ} E^0_\text{occ} + C^0_\text{occ} E^1_\text{occ}$$
Multiplying $C^{0,\dagger}_\text{occ}$ to both sides of the above equation, and knowing the occupied first-order response orbitals are orthogonal to the zeroth order, we get
$$E^1_\text{occ} = C^{0,\dagger}_\text{occ} H^1 C^{0}_\text{occ}$$

Substituting the above equation back to the equation earlier, and notice that $C^{0}_\text{occ}C^{0,\dagger}_\text{occ} = I - C^{0}_\text{vir}C^{0,\dagger}_\text{vir}$, we get
$$H^0C^1_\text{occ} - C^1_\text{occ}E^0_\text{occ} = -C^{0}_\text{vir}C^{0,\dagger}_\text{vir} H^1 C^0_\text{occ}$$

By multiplying $C^{0, \dagger}_\text{vir}$ from the left to both sides, and using $C^{0, \dagger}_\text{vir}H^0 = E^0_\text{vir} C^{0, \dagger}_\text{vir}$ we get 
$$C^1_\text{occ} = C^{0}_\text{vir} Z$$
where 
$$Z_{\mu\nu} = -\frac{\left(C^{0, \dagger}_\text{vir} H^1 C^{0}_\text{occ}\right)_{\mu\nu}}{E^0_{\text{vir},\mu} - E^0_{\text{occ},\nu}}$$

Finally, we derive the gradient form
$$\left.\frac{\partial D_{\text{HF}}}{\partial \delta }\right\vert_{\delta=0} = C^{0}_\text{occ} Z^\dagger C^{0,\dagger}_\text{vir} + C^{0}_\text{vir} Z C^{0,\dagger}_\text{occ} \\
\left.\frac{\partial D^\text{emb}_{\text{HF}}}{\partial \delta }\right\vert_{\delta=0} = R^\dagger \left.\frac{\partial D_{\text{HF}}}{\partial \delta }\right\vert_{\delta=0} R
$$

In our case, $H_0 = h_0 + u^\text{corr}$.
For each $u^\text{corr}_{\mu\nu}$, $H_1$ is the correlation potential $u^\text{corr}_{\mu\nu} = 1$ and the rest to be 0.

In [33]:
# evaluating the gradient 
orb_vir = mo_orbs[:, nelec:]
e_occ, e_vir = mo_e[:nelec], mo_e[nelec:]
# evaluate the gradient, a vector of the same size of uparams
grad = np.zeros(n_param)
E_diff_mat = (e_vir[:, None]-e_occ[None, :])
for i in range(n_param):
    uparam_tmp = np.zeros((n_param))
    uparam_tmp[i] = 1.
    du_corr_tmp = params2ucorr(uparam_tmp)
    du_corr_latt_tmp = imp2latt_1e(du_corr_tmp)
    Z = -np.dot(np.dot(orb_vir.T.conj(), du_corr_latt_tmp), orb_occ)/E_diff_mat
    drdm_latt = np.dot(np.dot(orb_occ, Z), orb_vir.T.conj()) + np.dot(np.dot(orb_occ, Z), orb_vir.T.conj())
    #drdm_emb = np.dot(np.dot(rotmat.T.conj(), drdm_latt), rotmat) # not the most efficient implementation, but straightforward
    drdm_imp = drdm_latt[:nimp, :nimp] # could be made simpler 
    grad[i] = -2*(rdm1_imp_diff*drdm_imp).sum()
print(grad)


[ 0.0017704  -0.01410449 -0.0017704 ]


In [34]:
# The easiest thing to check the gradient is to see if it makes things better
lr = 0.1 
u_params_new = u_params0 - lr * grad 
print(u_params_new)


[-0.00017704  0.00141045  0.00017704]
